<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [ ]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [151]:
import numpy as np
import time
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAWMILL CLASS**

In [171]:
class Inventory:
  def __init__(self, Av, Dist):
    self.Available = []
    self.Available.append(Av) #Dataframe that contains Available wood Inventory at diferent time t
    self.Dist = Dist #Matrix Distance from any cell to another cell
    self.Mills = [] #Dataframe that contains Id, typewood, type_product, and demand of mills
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the Refinery
    self.Harvested = [] #Ammount of harvested wood
    self.Residues = []  #Residues generated by harvesting
    self.Growth = [] #Growth ammount for each hexagon (feet cubic) 
    self.WoodType = 0 #type of wood needed (softwood, hardwood)
    self.ProdType = 0 #type of product generated (pulp or saw)
    self.Candidates=[] #Candidate location for biorefinery
    self.Refinery_Dem = 5000000  #Biorefinery Demand


  def Harvest_Mill(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')]
    a =self.Cell_Less(Id1,0,20)
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t, 20)
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat 
    ans = a

    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[0][self.Harvested[0]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[0]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[0].loc[a2,("Harvested")] = self.Harvested[0].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
    
    for j in range(len(ans)):                                                                         
      if not(self.Available[0][self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[0].loc[a2,("SumOfTotal")] = self.Available[0].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')]  
    

  def Harvest_All_Mills(self,t,k):
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill(t,i)
    self.Residues.append( Hardwood_Sawlog.Harvested[t].loc[:,('Unique_ID','Harvested')])
    #self.Residues.loc[t,('SumOfTotal')] =self.Residues.loc[t,('SumOfTotal')]*0.05
    self.Residues[0].loc[:,('Harvested')] = round(self.Residues[0].loc[:,('Harvested')]*0.05)
    print('Total Residues', self.Residues[0].loc[:,('Harvested')])

  
  def Inventory_Growth(self,t,rate = 0):
    if (rate == 0):
      r = self.Growth
    else:
      r = rate
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]*(1+r/100)

  def Cell_Less(self, Id_Mill, Time, Radio=20):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Cell_Less_Res(self, Id_Mill, Time, Radio=50):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Forest Residues available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Residues[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[0][self.Residues[0]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Disp_Radio(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
  

  def Disp_Radio_Res(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Residues Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Residues[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[0][self.Residues[0]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    return(Disp)


  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    #print("Numero de celdas ",len(Ans))
    return(len(Ans))


  def Optimal_Location(self):
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization
    Num_Cand = len(self.Candidates)
    Cost=[]
    for i in range(10):
      Id1 =self.Candidates[i]
      a =self.Cell_Less_Res(Id1,0,20)
      self.Dist_Refi = a.loc[:, ('DEST','DIST')]
      Num = len(self.Dist_Refi)
      H = m2.Array(m2.Var,(Num),value = 0)
      Dem = self.Refinery_Dem

      self.Available_Refi = self.Disp_Radio_Res( Id1, 0, 20)
      
      for i in range(Num):
        H[i].lower = 0
        H[i].upper = self.Available_Refi[i]
        
      prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
      
      m2.Equation(Dem ==m2.sum([H[i] for i in range(0,len(H))]))
      m2.Obj(prod0) 
      m2.solve(disp=False,debug=False)
      a = self.Dist_Refi
      flat = [item for l in H for item in l]
      a['Harvested'] = flat

      res = round(np.dot(flat,self.Dist_Refi.loc[:,('DIST')]))
      Cost.append(res)
    print('Cost', Cost)
    print('Minimo en:', Cost.index(min(Cost)))

    

    


**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [172]:
# Main Program
start_time = time.time()
#Upload distances from each hexagon to other
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Mills_Data.Demand = Mills_Data.Demand*83.3

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
v1 = Hardwood_Sawlog.Available[0]["Unique_ID"]==54549
v2 =int(Hardwood_Sawlog.Available[0].loc[v1,("SumOfTotal")])
r1 = len(Hardwood_Sawlog.Available[0])
Id_li = list(Hardwood_Sawlog.Available[0].loc[:,("Unique_ID")])
Ha_li =[0]*r1
Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
Hardwood_Sawlog.Harvested.append(pd.DataFrame(Data))
Num_Mills = 5
ma = Dist_Data['ORIG'].unique()
Hardwood_Sawlog.Candidates = ma

for k in range(Num_Mills):
  Mills_Data.loc[k,('Unique_ID')] = ma[k]
  
Hardwood_Sawlog.Mills=Mills_Data
Hardwood_Sawlog.Harvest_All_Mills(0,Num_Mills)  
Hardwood_Sawlog.Optimal_Location()
#print('AFTER HARVEST',Hardwood_Sawlog.Available[0].loc[v1,("SumOfTotal")])

Hardwood_Sawlog.Harvested[0].to_csv('/content/drive/My Drive/Colab Notebooks/out.csv')
print ("My program took", time.time() - start_time, "to run")

Cosechando Sawmill: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Cosechando Sawmill: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Cosechando Sawmill: 2


...

Cosechando Sawmill: 4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Total Residues 0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
          ...   
7758         0.0
7759         0.0
7760         0.0
7761         0.0
7762    113410.0
Name: Harvested, Length: 7763, dtype: float64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Cost [44144763, 44332284, 44881604, 54241698, 51961617, 49291726, 0, 56185039, 0, 0]
Minimo en: 6
My program took 112.17885899543762 to run
